# <font color='Red'>DDB (DESHIMA DATABASE)</font>

## Import Modules

In [1]:
from modules import *

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

## Make KIDs design table

In [3]:
chipname = {0:'LT119_FB3.3G_49ch'}

def make_design(infile):
    
    d_dict = design_dict()
    d_dict['hdr_val_lis'][1] = os.path.basename(infile)
    
    ##### read csv file
    kid_design = ascii.read(infile)
    #print( kid_design )
    
    ##### fill dictionary
    pixelid = 0
    chip = chipname[pixelid]
    nkid = len(kid_design['master_id'])

    d_dict['hdr_key_lis'].append('PIXEL%d' %pixelid)
    d_dict['hdr_val_lis'].append( chip )
    d_dict['hdr_com_lis'].insert(2, 'name of pixel %d' %pixelid)
    d_dict['hdr_key_lis'].append('NKID%d' %pixelid)
    d_dict['hdr_val_lis'].append( nkid )
    d_dict['hdr_com_lis'].insert(3, 'number of KIDs (pixel %d)' %pixelid)
    
    d_dict['cols_data_lis'].append( np.ones(nkid)*pixelid )
    d_dict['cols_data_lis'].append( list(kid_design['master_id']) )
    d_dict['cols_data_lis'].append( list(kid_design['attribute']) )
    d_dict['cols_data_lis'].append( list(kid_design['f_KID_design']) )   
    d_dict['cols_data_lis'].append( list(kid_design['F_filter_design']) )

    return createBinTableHdu(d_dict)

In [4]:
d_hdu = make_design('sqlite/LT119_FB3.3G_49ch.csv')

/Users/karatsu/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/astropy/io/fits/column.py:569: UserWarning: Warning: converting a masked element to nan.
  array = np.array(array)


In [5]:
print(d_hdu.header)
table.Table(d_hdu.data)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   20 / length of dimension 1                          NAXIS2  =                   66 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    5 / number of table fields                         EXTNAME = 'KIDDES  '           / name of binary data                            FILENAME= 'LT119_FB3.3G_49ch.csv' / input filename                              PIXEL0  = 'LT119_FB3.3G_49ch'  / name of pixel 0                                NKID0   =                   66 / number of KIDs (pixel 0)                       TTYPE1  = 'pixelid '           / label f

pixelid,masterid,attribute,fr_design,F_filter_design
int16,int16,str8,float32,float32
0,0,wideband,5.2,nan
0,1,wideband,5.22,nan
0,2,wideband,5.24,nan
0,3,wideband,5.26,nan
0,4,wideband,5.28,nan
0,5,wideband,5.3,nan
0,6,wideband,5.32,nan
0,10,blind,6.6,nan
0,11,blind,6.62,nan


In [6]:
kidname = {}
for i,j in zip(d_hdu.data['masterid'],d_hdu.data['attribute']):
    kidname[i] = j
print( len(kidname), kidname )

66 {0: 'wideband', 1: 'wideband', 2: 'wideband', 3: 'wideband', 4: 'wideband', 5: 'wideband', 6: 'wideband', 10: 'blind', 11: 'blind', 12: 'blind', 13: 'blind', 14: 'blind', 15: 'blind', 20: 'Al', 21: 'Al', 30: 'NbTiN', 31: 'NbTiN', 100: 'filter', 107: 'filter', 114: 'filter', 121: 'filter', 128: 'filter', 135: 'filter', 142: 'filter', 101: 'filter', 108: 'filter', 115: 'filter', 122: 'filter', 129: 'filter', 136: 'filter', 143: 'filter', 102: 'filter', 109: 'filter', 116: 'filter', 123: 'filter', 130: 'filter', 137: 'filter', 144: 'filter', 103: 'filter', 110: 'filter', 117: 'filter', 124: 'filter', 131: 'filter', 138: 'filter', 145: 'filter', 104: 'filter', 111: 'filter', 118: 'filter', 125: 'filter', 132: 'filter', 139: 'filter', 146: 'filter', 105: 'filter', 112: 'filter', 119: 'filter', 126: 'filter', 133: 'filter', 140: 'filter', 147: 'filter', 106: 'filter', 113: 'filter', 120: 'filter', 127: 'filter', 134: 'filter', 141: 'filter', 148: 'filter'}


## Make KIDs filter table

In [7]:
import re
import json

def make_filter(dirname, ffile, jfile):

    runid = int( re.split('_', dirname)[-2] )
    framelen = 19 # depende on runid
    pixelid = 0    

    (kidid, Fsubmm, Spectrum_df, refid, F, dF, Q, dQ, fit_params)\
        = np.load(os.path.join(dirname, ffile), encoding='bytes')
    kiddict = json.load( open(os.path.join(dirname, jfile)) )
    print( kiddict )
    
    masterid = []
    Fsubmm_, Spectrum_df_ = [], []
    nkid = len(kidid)
    for i in range(nkid):
        tmpid = kidid[i]
        
        try:
            mid = list(kiddict.keys())[ list(kiddict.values()).index(tmpid) ]
            mid = int(mid)
            kind = kidname[mid]
        except ValueError:
            print(i)
            mid = -1
            kind = 'unknown'

        masterid.append( mid )
        Fsubmm_.append(Fsubmm[i])
        Spectrum_df_.append(Spectrum_df[i])
    
    f_dict = filter_dict()
    f_dict['hdr_val_lis'][1] = os.path.basename(ffile)
    f_dict['hdr_val_lis'][2] = os.path.basename(jfile)
    
    f_dict['hdr_key_lis'].append('NKID%d' %pixelid)
    f_dict['hdr_val_lis'].append( nkid )
    f_dict['hdr_com_lis'].insert(3, 'number of KIDs (pixel %d)' %pixelid)
    
    #####
    f_dict['cols_data_lis'].append( np.ones(nkid)*pixelid )
    f_dict['cols_data_lis'].append( kidid )
    f_dict['cols_data_lis'].append( masterid )
    runid_ = np.ones(nkid)*runid
    framelen_ = np.ones(nkid)*framelen
    refid_ = np.ones(nkid)*refid
    f_dict['cols_data_lis'].append( np.array([runid_, framelen_, refid_]).T )
    f_dict['cols_data_lis'].append( np.array([F, dF]).T )
    f_dict['cols_data_lis'].append( np.array([Q, dQ]).T )
      
    f_dict['cols_data_lis'].append( fit_params )       
    f_dict['cols_data_lis'].append( Fsubmm_ )
    f_dict['cols_data_lis'].append( Spectrum_df_ )
    f_dict['tform'][7] = '%dE' %len(Fsubmm_[0])
    f_dict['tform'][8] = '%dE' %len(Spectrum_df_[0])
    
    
    return createBinTableHdu(f_dict)   

In [8]:
f_hdu = make_filter(dirname='TerahertzScan_20170731_165125_00695_ana',
                    ffile='AnaSpectrum2nd_fit_for_DDB.npy',
                    jfile='kid_corresp.json')

{'0': 0, '1': 2, '2': 4, '3': 1, '4': 3, '5': 5, '6': 6, '10': 57, '11': 58, '12': 59, '13': 60, '14': 61, '15': 62, '128': 35, '129': 36, '130': 38, '131': 37, '132': 39, '133': 40, '134': 41, '135': 42, '136': 43, '137': 45, '138': 46, '139': 47, '140': 48, '141': 49, '142': 50, '143': 51, '144': 52, '145': 53, '146': 54, '147': 55, '148': 56, '100': 7, '101': 9, '102': 8, '103': 10, '104': 11, '105': 12, '106': 13, '107': 14, '108': 17, '109': 18, '110': 15, '111': 16, '112': 19, '113': 20, '114': 21, '115': 23, '116': 25, '117': 24, '118': 22, '119': 26, '120': 27, '121': 28, '122': 29, '123': 30, '124': 32, '125': 31, '126': 33, '127': 34}
44


In [9]:
print(f_hdu.header)
table.Table(f_hdu.data)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                16044 / length of dimension 1                          NAXIS2  =                   63 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    9 / number of table fields                         EXTNAME = 'KIDFILT '           / name of binary data                            FILENAME= 'AnaSpectrum2nd_fit_for_DDB.npy' / input data filename                JSONNAME= 'kid_corresp.json'   / input json filename                            NKID0   =                   63 / number of KIDs (pixel 0)                       TTYPE1  = 'pixelid '           / label f

pixelid,kidid,masterid,"runid, framelen, refid [3]","F_filter, dF_filter [2]","Q_filter, dQ_filter [2]",fit params [6],Raw Toptica F [1999],Raw df resp. [1999]
int16,int16,int16,int16,float32,float32,float32,float32,float32
0,0,0,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,5.31228e-08 .. 1.12719e-08
0,1,3,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,4.777e-08 .. 6.65043e-09
0,2,1,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,5.96409e-08 .. 5.90139e-09
0,3,4,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,6.99581e-08 .. 5.5669e-09
0,4,2,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,5.84056e-08 .. 6.89835e-09
0,5,5,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,5.81362e-08 .. 4.80009e-09
0,6,6,695 .. 0,nan .. nan,nan .. nan,nan .. nan,300.0 .. 399.9,4.2877e-08 .. 4.93584e-09
0,7,100,695 .. 0,376.158 .. 0.00387784,287.354 .. 2.41789,95.0547 .. 0.0110147,300.0 .. 399.9,1.053e-08 .. 1.07848e-08
0,8,102,695 .. 0,363.022 .. 0.00799948,161.062 .. 1.6173,15.3692 .. 0.0226326,300.0 .. 399.9,9.88716e-09 .. 8.2716e-09


In [10]:
print(f_hdu.data[44])

(0, 44, -1, array([695,  19,   0], dtype=int16), array([ nan,  nan], dtype=float32), array([ nan,  nan], dtype=float32), array([ nan,  nan,  nan,  nan,  nan,  nan], dtype=float32), array([ 300.        ,  300.04998779,  300.1000061 , ...,  399.79998779,
        399.8500061 ,  399.8999939 ], dtype=float32), array([  5.27341548e-08,   1.30950795e-07,   1.00359962e-07, ...,
         9.04725823e-08,   8.64671179e-08,   8.11201204e-08], dtype=float32))


## Make KIDs repsonsivity table

In [11]:
def make_resp(dirname, rfile, jfile):

    runid = int( re.search('([0-9]+)', dirname).group() )
    framelen = 19 # depende on runid
    pixelid = 0
    
    ##### phase = fshift
    (kidid, Tarray, Parray, dParray,\
        data_ampl, err_ampl, data_phase, err_phase,\
         fit_ampl, fiterr_ampl, fit_phase, fiterr_phase) = np.load(os.path.join(dirname, rfile), encoding='bytes')
    kiddict = json.load( open(jfile) )
    print( kiddict )
    
    masterid = []
    fit_params = []
    Tload, Pload, df, dferr = [], [], [], []
    nkid = len(kidid)
    for i in range(nkid):
        tmpid = kidid[i]
        
        try:
            mid = list(kiddict.keys())[ list(kiddict.values()).index(tmpid) ]
            mid = int(mid)
            kind = kidname[mid]
        except ValueError:
            print(i)
            mid = -1
            kind = 'unknown'

        masterid.append( mid )
        fit_params.append( [fit_phase[i][0], fiterr_phase[i][0], fit_phase[i][1], fiterr_phase[i][1]] )
        Tload.append( Tarray[i] )
        Pload.append( Parray[i] )
        df.append( data_phase[i] )
        dferr.append( err_phase[i] )
    
    r_dict = resp_dict()
    r_dict['hdr_val_lis'][1] = os.path.basename(rfile)
    r_dict['hdr_val_lis'][2] = os.path.basename(jfile)
    
    r_dict['hdr_key_lis'].append('NKID%d' %pixelid)
    r_dict['hdr_val_lis'].append( nkid )
    r_dict['hdr_com_lis'].insert(3, 'number of KIDs (pixel %d)' %pixelid)
    
    #####
    r_dict['cols_data_lis'].append( np.ones(nkid)*pixelid )
    r_dict['cols_data_lis'].append( kidid )
    r_dict['cols_data_lis'].append( masterid )
    runid_ = np.ones(nkid)*runid
    framelen_ = np.ones(nkid)*framelen
    r_dict['cols_data_lis'].append( np.array([runid_, framelen_]).T )
      
    r_dict['cols_data_lis'].append( fit_params )
    
    r_dict['cols_data_lis'].append( Tload )
    r_dict['cols_data_lis'].append( Pload )
    r_dict['cols_data_lis'].append( df )
    r_dict['cols_data_lis'].append( dferr )
    
    r_dict['tform'][5] = '%dE' %len(Tload[0])
    r_dict['tform'][6] = '%dE' %len(Pload[0])
    r_dict['tform'][7] = '%dE' %len(df[0])
    r_dict['tform'][8] = '%dE' %len(dferr[0])
    
    return createBinTableHdu(r_dict)   

In [12]:
resp_hdu = make_resp(dirname='Basic707_Grid_ana',
                     rfile='SetSlice/Responsivity_fit.npy',
                     jfile='TerahertzScan_20170731_165125_00695_ana/kid_corresp.json')

{'0': 0, '1': 2, '2': 4, '3': 1, '4': 3, '5': 5, '6': 6, '10': 57, '11': 58, '12': 59, '13': 60, '14': 61, '15': 62, '128': 35, '129': 36, '130': 38, '131': 37, '132': 39, '133': 40, '134': 41, '135': 42, '136': 43, '137': 45, '138': 46, '139': 47, '140': 48, '141': 49, '142': 50, '143': 51, '144': 52, '145': 53, '146': 54, '147': 55, '148': 56, '100': 7, '101': 9, '102': 8, '103': 10, '104': 11, '105': 12, '106': 13, '107': 14, '108': 17, '109': 18, '110': 15, '111': 16, '112': 19, '113': 20, '114': 21, '115': 23, '116': 25, '117': 24, '118': 22, '119': 26, '120': 27, '121': 28, '122': 29, '123': 30, '124': 32, '125': 31, '126': 33, '127': 34}
44


In [13]:
print(resp_hdu.header)
table.Table(resp_hdu.data)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  218 / length of dimension 1                          NAXIS2  =                   63 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    9 / number of table fields                         EXTNAME = 'KIDRESP '           / name of binary data                            FILENAME= 'Responsivity_fit.npy' / input data filename                          JSONNAME= 'kid_corresp.json'   / input json filename                            NKID0   =                   63 / number of KIDs (pixel 0)                       TTYPE1  = 'pixelid '           / label f

pixelid,kidid,masterid,"runid, framelen [2]",fit params [4],Raw Tload [12],Raw Pload [12],Raw df resp. [12],Raw dferr [12]
int16,int16,int16,int16,float32,float32,float32,float32,float32
0,0,0,707 .. 19,1.12175e-06 .. 5.8831e-08,77.0 .. 295.0,2.23071e-11 .. 6.63818e-11,-3.43749e-06 .. 5.98405e-06,1.16312e-08 .. 1.28881e-08
0,1,3,707 .. 19,4.28176e-07 .. 3.55343e-08,77.0 .. 295.0,1.16959e-11 .. 3.48278e-11,-3.02605e-06 .. 5.56803e-07,1.1991e-08 .. 1.22471e-08
0,2,1,707 .. 19,1.13931e-06 .. 6.19285e-08,77.0 .. 295.0,2.15396e-11 .. 6.41049e-11,-3.27228e-06 .. 6.28223e-06,1.02434e-08 .. 1.07853e-08
0,3,4,707 .. 19,4.2865e-07 .. 3.64856e-08,77.0 .. 295.0,9.707e-12 .. 2.89054e-11,8.21626e-06 .. 1.18001e-05,1.08061e-08 .. 9.62984e-09
0,4,2,707 .. 19,1.19689e-06 .. 7.13392e-08,77.0 .. 295.0,2.03637e-11 .. 6.06041e-11,2.47615e-05 .. 3.47882e-05,1.2634e-08 .. 1.42527e-08
0,5,5,707 .. 19,2.97456e-07 .. 2.86076e-08,77.0 .. 295.0,9.16402e-12 .. 2.72883e-11,1.96855e-05 .. 2.21706e-05,1.28616e-08 .. 9.71301e-09
0,6,6,707 .. 19,2.15536e-07 .. 2.38698e-08,77.0 .. 295.0,8.74722e-12 .. 2.60425e-11,-1.42993e-05 .. -1.25009e-05,1.34233e-08 .. 1.03703e-08
0,7,100,707 .. 19,1.39323e-06 .. 8.96586e-08,77.0 .. 295.0,7.97035e-14 .. 2.36409e-13,-2.25001e-05 .. -1.08154e-05,1.86545e-08 .. 1.86217e-08
0,8,102,707 .. 19,1.69067e-06 .. 3.55977e-07,77.0 .. 295.0,8.42903e-13 .. 2.50324e-12,-7.06318e-06 .. 7.33461e-06,1.88222e-08 .. 1.61918e-08


## Make KIDs RCP table

In [17]:
def make_rcp():
#    filters_data = Path(rcp_data).expanduser()

    pixelid   = [0]
    offsetaz  = np.zeros(1)
    offsetel  = np.zeros(1)
    gain      = np.ones(1)
    
    filename = None

    r_dict = rcp_dict()

    r_dict['hdr_val_lis'][1] = filename
    r_dict['cols_data_lis'] = [pixelid, offsetaz, offsetel, gain]

    return createBinTableHdu(r_dict)

In [18]:
rcp_hdu = make_rcp()

In [19]:
print(rcp_hdu.header)
table.Table(rcp_hdu.data)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   32 / length of dimension 1                          NAXIS2  =                    1 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    4 / number of table fields                         EXTNAME = 'KIDRCP  '           / name of binary table                           FILENAME= '' / filename??                                                       TTYPE1  = 'pixelid '                                                            TFORM1  = 'K       '           / label for field 0                              TTYPE2  = 'offsetaz'           / data fo

pixelid,offsetaz,offsetel,gain
int64,float64,float64,float64
0,0.0,0.0,1.0


## Save to fits file

In [20]:
hdus = fits.HDUList()
hdus.append( fits.PrimaryHDU() )

hdus['PRIMARY'].header['DDB_ID'] = '20171031', 'The ID of DESHIMA DataBase'
hdus.append( d_hdu )
hdus.append( f_hdu )
hdus.append( resp_hdu )
hdus.append( rcp_hdu )

In [21]:
hdus.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1  KIDDES        1 BinTableHDU     24   66R x 5C   ['I', 'I', '8A', 'E', 'E']   
  2  KIDFILT       1 BinTableHDU     32   63R x 9C   ['I', 'I', 'I', '3I', '2E', '2E', '6E', '1999E', '1999E']   
  3  KIDRESP       1 BinTableHDU     32   63R x 9C   ['I', 'I', 'I', '2I', '4E', '12E', '12E', '12E', '12E']   
  4  KIDRCP        1 BinTableHDU     21   1R x 4C   ['K', 'D', 'D', 'D']   


In [21]:
ddb_fitsname = 'DDB_' + hdus['PRIMARY'].header['DDB_ID'] + '.fits'
hdus.writeto(ddb_fitsname, overwrite=True)